In [ ]:
from flask import Flask, jsonify, request
import pandas as pd
from sklearn.ensemble import IsolationForest  # 예시용 Isolation Forest 모델 사용

# Flask 애플리케이션 초기화
app = Flask(__name__)

# 간단한 이상 탐지 모델 생성 (테스트용)
#model = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)
# 임의의 데이터로 모델 학습 (실제 서비스에서는 적절한 데이터를 사용해야 합니다)
#model.fit([[0, 0], [1, 1], [2, 2]])

# 데이터 전처리 함수 (샘플)
def preprocess_data(df):
    # 필요한 컬럼 선택 및 인코딩 (여기서는 단순히 넘파이 배열로 변환)
    return df.values

# 실시간 로그 데이터 처리 및 예측 엔드포인트
@app.route('/monitor', methods=['POST'])
def monitor():
    try:
        # POST 요청으로부터 JSON 데이터 수신
        log = request.get_json()
        if not log:
            return jsonify({"error": "No JSON data received"}), 400
        
        # 데이터프레임으로 변환
        df = pd.DataFrame([log])
        processed_log = preprocess_data(df)  # 전처리 수행

        # 모델을 사용한 이상 탐지 수행
        anomaly_score = model.decision_function(processed_log)
        is_anomaly = model.predict(processed_log)

        # 응답 생성
        response = {
            "anomaly_score": float(anomaly_score[0]),  # JSON 직렬화 가능하도록 변환
            "is_anomaly": bool(is_anomaly[0] == -1)  # -1이면 이상
        }
        return jsonify(response), 200

    except Exception as e:
        # 예외 처리 및 오류 응답 반환
        return jsonify({"error": str(e)}), 500

# 서버 실행
if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000, use_reloader=False)
